In [8]:
# 📁 02_preprocessing_labeling.ipynb
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
import os

# 1. 경로 설정
news_path = "../data/naver_realestate_news.csv"
label_path = "../data/labeled_news.xlsx"
save_path = "../data/preprocessed_labeled.csv"

# 2. 경로 존재 확인
if not os.path.exists(news_path):
    raise FileNotFoundError(f"뉴스 파일을 찾을 수 없습니다: {news_path}")
if not os.path.exists(label_path):
    raise FileNotFoundError(f"라벨 파일을 찾을 수 없습니다: {label_path}")

# 3. 데이터 로딩
df = pd.read_csv(news_path)
labeled_df = pd.read_excel(label_path)
okt = Okt()

# 4. 전처리 함수 정의
def clean_text(text):
    text = str(text)
    text = re.sub(r"[^가-힣0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

stopwords = ["의", "가", "이", "은", "들", "는", "좀", "잘", "걍", "과", "도", "를", "으로", "자", "에", "와", "한", "하다"]

def tokenize(text):
    try:
        return [word for word in okt.morphs(text) if word not in stopwords]
    except Exception as e:
        print(f"Tokenization error: {e}")
        return []

# 5. 텍스트 정제 및 형태소 분석
df.dropna(subset=["title"], inplace=True)
df["clean_title"] = df["title"].apply(clean_text)
df["tokens"] = df["clean_title"].apply(tokenize)

# 6. 수동 라벨 추가 (데이터 불균형 완화 목적)
manual = pd.DataFrame([
    {"title": "서울 아파트 분양 완판, 실수요자 몰려", "label": 1},
    {"title": "정부 부동산 대책 효과 기대", "label": 1},
    {"title": "전세 사기 피해자 속출, 부동산 시장 불신", "label": 0},
    {"title": "집값 하락세 지속, 투자자들 손실 우려", "label": 0}
])
labeled_df = pd.concat([labeled_df, manual], ignore_index=True)

# 7. 라벨 정제 및 병합
labeled_df = labeled_df[["title", "label"]].dropna()
labeled_df["title"] = labeled_df["title"].apply(clean_text)

merged = pd.merge(df, labeled_df, on="title", how="inner")
merged.drop_duplicates(subset=["title"], inplace=True)
merged = merged[merged["label"].isin([0, 1, 2])]

# 8. 저장
if not os.path.isdir(os.path.dirname(save_path)):
    os.makedirs(os.path.dirname(save_path))

merged.to_csv(save_path, index=False, encoding="utf-8-sig")

# 9. 요약 출력 (이모지 제거)
print("[전처리 완료]")
print("[전체 샘플 수]", len(merged))
print("[라벨 분포]\n", merged["label"].value_counts())
print("[저장 경로]", save_path)


[전처리 완료]
[전체 샘플 수] 50
[라벨 분포]
 label
2    23
0    18
1     9
Name: count, dtype: int64
[저장 경로] ../data/preprocessed_labeled.csv
